In [3]:
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
from PIL import Image
import pandas as pd
import numpy as np
import re
import io
import sqlite3


2024-12-08 13:48:13.669 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [4]:
def image_to_text(path):
    
    input_image = Image.open(path)

    #connvert image to array

    image_arr = np.array(input_image)

    reader = easyocr.Reader(['en'])
    text = reader.readtext(image_arr, detail=0)
    
    return text, input_image


In [5]:
text_image, input_image = image_to_text("C:/GUVI PROJECTS/PHONE PE PULSE/.venv/1.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [33]:
text_image, input_image = image_to_text("C:/GUVI PROJECTS/PHONE PE PULSE/.venv/2.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [34]:
text_image, input_image = image_to_text("C:/GUVI PROJECTS/PHONE PE PULSE/.venv/3.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [35]:
text_image, input_image = image_to_text("C:/GUVI PROJECTS/PHONE PE PULSE/.venv/4.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [36]:
text_image, input_image = image_to_text("C:/GUVI PROJECTS/PHONE PE PULSE/.venv/5.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
input_image

In [7]:
text_image

['Selva',
 'DATA MANAGER',
 '+123-456-7890',
 '+123-456-7891',
 'WWW XYZI.com',
 'hello@XYZ1.com',
 '123 ABC St , Chennai;',
 'selva',
 'TamilNadu 600113',
 'digitals']

In [8]:
def extracted_text(texts):
    
    extracted_dict = {"NAME":[], "DESIGNATION":[], "COMPANY_NAME":[], "CONTACT":[],
                      "EMAIL":[], "WEBSITE":[], "ADDRESS":[], "PINCODE":[]}
    extracted_dict["NAME"].append(texts[0])
    extracted_dict["DESIGNATION"].append(texts[1])
    
    for i in range(2,len(texts)):
        
        if texts[i].startswith("+") or (texts[i].replace("-","").isdigit() and "-" in texts[i]):
            
            extracted_dict["CONTACT"].append(texts[i])
            
        elif "@" in texts[i] and ".com" in texts[i]:
           
           extracted_dict["EMAIL"].append(texts[i]) 
           
        elif "WWW" in texts[i] or  "www" in texts[i] or "Www" in texts[i] or  "wWw" in texts[i] or "wwW" in texts[i]:
            
            small = texts[i].lower()
            extracted_dict["WEBSITE"].append(small)
            
        elif "Tamil Nadu" in texts[i] or "TamilNadu" in texts[i] or "TN" in texts[i] or texts[i].isdigit():
            
            extracted_dict["PINCODE"].append(texts[i])
            
        elif re.match(r'^[A-Za-z]', texts[i]):
            
            extracted_dict["COMPANY_NAME"].append(texts[i])
            
        else:
            remove_colon = re.sub(r'[,;]', '', texts[i])
            extracted_dict["ADDRESS"].append(remove_colon)     
            
    for key,value in extracted_dict.items():  
        #print(key,":", value, len(value))     
        if len(value)>0:
            concadenate = " ".join(value)
            extracted_dict[key] = [concadenate]
            
        else:
            value = "NA"
            extracted_dict[key] = [value]
     
    return extracted_dict

In [9]:
text_data = extracted_text(text_image)

In [10]:
df = pd.DataFrame(text_data)
df

,NAME,DESIGNATION,COMPANY_NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE
0,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113


In [11]:
# converting image to bytes

Image_bytes = io.BytesIO()
input_image.save(Image_bytes, format= "PNG")

image_data = Image_bytes.getvalue()
image_data


#creating dictionary

data = {"IMAGE":[image_data]}
data

df_1 = pd.DataFrame(data)

concat_df = pd.concat([df,df_1], axis=1)
concat_df

,NAME,DESIGNATION,COMPANY_NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE,IMAGE
0,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


In [12]:
# sql connection

mydb = sqlite3.connect("bizcardx.db")
cursor = mydb.cursor()

#Table creation

create_table_query = '''CREATE TABLE IF NOT EXISTS bizcardx_details(name VARCHAR(255),
                                                                    designation VARCHAR(255),
                                                                    company_name VARCHAR(255),
                                                                    contact VARCHAR(255),
                                                                    email VARCHAR(255),
                                                                    website TEXT,
                                                                    address TEXT,
                                                                    pincode VARCHAR(255),
                                                                    image text)'''
                                                                    
cursor.execute(create_table_query)
mydb.commit()

In [13]:
#insert queries

insert_query = '''INSERT INTO bizcardx_details(name, designation, company_name, contact, email, 
                                               website, address, pincode, image )
                                               
                                               VALUES(?,?,?,?,?,?,?,?,?)'''
                                            
datas = concat_df.values.tolist()[0]
cursor.execute(insert_query, datas)
mydb.commit()
                                               
                                               

In [17]:
#select query 

mydb = sqlite3.connect("bizcardx.db")
cursor = mydb.cursor()

select_query = "SELECT NAME FROM bizcardx_details"

cursor.execute(select_query)
table1 = cursor.fetchall()
mydb.commit()

names = []

for i in table1:
    names.append(i[0])
    
names

['Selva',
 'SANTHOSH',
 'SANTHOSH',
 'SANTHOSH',
 'SANTHOSH',
 'Selva',
 'Selva',
 'Selva']